# Azure AI Agentai su Model Context Protocol (MCP) palaikymu

Šiame užrašų knygelėje demonstruojama, kaip naudoti Azure AI Agentus su Model Context Protocol (MCP) įrankiais, siekiant sukurti intelektualų agentą, galintį pasinaudoti išoriniais MCP serveriais, kad padidintų savo galimybes.


## Įdiekite reikalingus „NuGet“ paketus

Pirmiausia turime įdiegti „Azure AI Agents Persistent“ paketą, kuris suteikia pagrindinę funkcionalumą darbui su „Azure AI Agents“.


## Privalumai naudojant autentifikaciją be raktų

Ši užrašų knygelė demonstruoja **autentifikaciją be raktų**, kuri suteikia keletą privalumų:
- ✅ **Nereikia valdyti API raktų** - Naudojama autentifikacija, pagrįsta Azure identitetu
- ✅ **Padidintas saugumas** - Nėra saugomų slaptažodžių kode ar konfigūracijoje
- ✅ **Automatinė kredencialų rotacija** - Azure tvarko kredencialų gyvavimo ciklą
- ✅ **Prieiga pagal vaidmenis** - Naudojama Azure RBAC smulkiam leidimų valdymui

`DefaultAzureCredential` automatiškai naudos geriausią prieinamą kredencialų šaltinį:
1. Valdomą identitetą (kai vykdoma Azure aplinkoje)
2. Azure CLI kredencialus (kuriant programą)
3. Visual Studio kredencialus
4. Aplinkos kintamuosius (jei sukonfigūruota)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Įdiekite „Azure Identity“ paketą autentifikacijai su „Azure“ paslaugomis naudojant „DefaultAzureCredential“.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Importuokite reikalingas vardų sritis

Importuokite būtinas vardų sritis, skirtas Azure AI agentams ir Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Konfigūruoti Azure AI Agent Client (Autentifikacija be rakto)

Nustatykite konfigūracijos kintamuosius ir sukurkite PersistentAgentsClient naudodami **autentifikaciją be rakto**:
- **projectEndpoint**: Azure AI Foundry projekto pabaigos taškas
- **modelDeploymentName**: Diegto AI modelio pavadinimas (GPT-4.1 nano)
- **mcpServerUrl**: MCP serverio URL (Microsoft Learn API)
- **mcpServerLabel**: Etiketė MCP serveriui identifikuoti
- **DefaultAzureCredential**: Naudoja valdomą tapatybę, Azure CLI ar kitus kredencialų šaltinius (API raktai nereikalingi)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Sukurti MCP Įrankio Apibrėžimą

Sukurkite MCP įrankio apibrėžimą, kuris prisijungtų prie Microsoft Learn MCP serverio. Tai leis agentui pasiekti Microsoft Learn turinį ir dokumentaciją.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Sukurkite AI agentą

Sukurkite nuolatinį AI agentą su nurodytu modeliu ir MCP įrankiais. Agentas sukonfigūruotas taip:
- GPT-4.1 nano modelis
- Instrukcijos naudoti MCP įrankius pagalbai
- Prieiga prie Microsoft Learn MCP serverio


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Sukurti pokalbio giją ir išsiųsti žinutę

Sukurkite pokalbio giją ir išsiųskite vartotojo žinutę, klausdami apie skirtumą tarp Azure OpenAI ir OpenAI. Tai patikrins agento gebėjimą naudoti MCP įrankius, kad pateiktų tikslią informaciją.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Konfigūruoti MCP įrankio išteklius (be rakto)

Nustatykite MCP įrankio išteklius. Norint visiškai atsisakyti rakto, galite pašalinti pasirinktines antraštes, jei MCP serveris palaiko autentifikaciją, pagrįstą „Azure“ tapatybe. Žemiau pateiktas pavyzdys rodo, kaip pridėti antraštes, jei reikia, tačiau be rakto scenarijams jos gali būti nereikalingos.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Pradėti agento vykdymą

Sukurkite ir pradėkite vykdymą, kad apdorotumėte vartotojo žinutę. Agentas naudos sukonfigūruotus MCP įrankius ir išteklius atsakymui generuoti.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Stebėkite vykdymą ir tvarkykite įrankių patvirtinimus (be rakto)

Stebėkite agento vykdymo būseną ir tvarkykite visus reikalingus įrankių patvirtinimus. Ši ciklo eiga:
1. Laukia, kol vykdymas bus baigtas arba reikės veiksmų
2. Automatiškai patvirtina MCP įrankių užklausas, kai to reikia
3. Naudojant autentifikaciją be rakto, antraštės gali būti nereikalingos, jei MCP serveris palaiko „Azure“ identitetą


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Rodyti Pokalbio Rezultatus

Gaukite ir parodykite visas žinutes iš pokalbio gijos, rodydami tiek vartotojo klausimą, tiek agento atsakymą. Žinutės pateikiamos chronologine tvarka su laiko žymomis ir vaidmenų indikatoriais.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama naudoti profesionalų žmogaus vertimą. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus interpretavimus, atsiradusius dėl šio vertimo naudojimo.
